In [33]:
import pandas as pd
import numpy as np
import math
import os
import json

import viewdb_to_bq  as x

import incident_data_pb2 as pb2_incident


from google.protobuf.timestamp_pb2 import Timestamp

from datetime import datetime ,timezone

# Parameter

In [34]:
view_name="xyz_incident"
view_name_id="incident_id"
datetimeCols=["open_datetime","close_datetime"]

projectId="pongthorn"
main_dataset_id='SMartDataAnalytics'
table_name="incident"

dt_imported=datetime.now(timezone.utc) # utc
dt_imported=datetime.strptime(dt_imported.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
dt_imported

datetime.datetime(2024, 1, 17, 14, 20, 41)

# Init Const and Variable 

# BQ-Storage-API Data Transformation

In [35]:
df=pd.read_csv("incident_bq-storage-api.csv")

In [36]:
print("Add timestamp import")
dtimestamp = Timestamp()
dtimestamp.FromDatetime(dt_imported)
update_at_micro_timestampe =dtimestamp.ToMicroseconds()
df['update_at']=update_at_micro_timestampe 


Add timestamp import


In [37]:
print("Change action type")

def change_action_merge_to_bq_storage_api(x):
    if x=="added" or x=="changed":
        return  "UPSERT"
    else:
        return "DELETE"   
df["_CHANGE_TYPE"]=df['action'].apply(change_action_merge_to_bq_storage_api)
df=df.drop(columns=['action'])

Change action type


In [38]:
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   incident_id     3 non-null      int64  
 1   inventory_id    2 non-null      float64
 2   incident_type   2 non-null      object 
 3   service_type    2 non-null      object 
 4   severity        2 non-null      object 
 5   status          2 non-null      object 
 6   open_datetime   2 non-null      object 
 7   close_datetime  2 non-null      object 
 8   update_at       3 non-null      int64  
 9   _CHANGE_TYPE    3 non-null      object 
dtypes: float64(1), int64(2), object(7)
memory usage: 368.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
0,4372,17269.0,Controller/Node Failure,Incident,Major,Closed,2024-01-15 06:16,2024-01-17 14:13,1705501241000000,UPSERT
1,4384,19610.0,Software,Incident,Critical,Closed,2024-01-17 14:08,2024-01-17 14:13,1705501241000000,UPSERT
2,4097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1705501241000000,DELETE


# Spit data into UPSERT and DELETE 

In [39]:
dfUpsert=df.query("_CHANGE_TYPE=='UPSERT'")
dfUpsert.info()
dfUpsert.tail()


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   incident_id     2 non-null      int64  
 1   inventory_id    2 non-null      float64
 2   incident_type   2 non-null      object 
 3   service_type    2 non-null      object 
 4   severity        2 non-null      object 
 5   status          2 non-null      object 
 6   open_datetime   2 non-null      object 
 7   close_datetime  2 non-null      object 
 8   update_at       2 non-null      int64  
 9   _CHANGE_TYPE    2 non-null      object 
dtypes: float64(1), int64(2), object(7)
memory usage: 176.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
0,4372,17269.0,Controller/Node Failure,Incident,Major,Closed,2024-01-15 06:16,2024-01-17 14:13,1705501241000000,UPSERT
1,4384,19610.0,Software,Incident,Critical,Closed,2024-01-17 14:08,2024-01-17 14:13,1705501241000000,UPSERT


In [24]:
dfDelete=df.query("_CHANGE_TYPE=='DELETE'")
dfDelete.info()
dfDelete

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   incident_id     0 non-null      int64 
 1   inventory_id    0 non-null      int64 
 2   incident_type   0 non-null      object
 3   service_type    0 non-null      object
 4   severity        0 non-null      object
 5   status          0 non-null      object
 6   open_datetime   0 non-null      object
 7   close_datetime  0 non-null      object
 8   update_at       0 non-null      int64 
 9   _CHANGE_TYPE    0 non-null      object
dtypes: int64(3), object(7)
memory usage: 0.0+ bytes


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE


# Upsert dataframe tranformation

# Timezone and UTC Convert
## If you convert any time of any tz to timestampe for converting to Microseconds , it wll turn into UTC  so DateTime is UTC
* to_char((abc.incident_datetime AT TIME ZONE 'Asia/Bangkok'::text),
           'YYYY-MM-DD HH24:MI'::text)   AS open_datetime
* to_char((abc.incident_datetime AT TIME ZONE 'UTC'::text),
               'YYYY-MM-DD HH24:MI'::text)   AS open_datetime
* https://www.epochconverter.com/




## WorkAround Sol: null dattime is replaced with 0(GMT:1-1-1970 12:00:00 AM)

In [25]:
if dfUpsert.empty==False:
    print("Convert strng to datetime and microseconds accordingly")

    def convert_string_to_datetime_timestamp_microseconds (dt_str):
        if isinstance(dt_str, str):
            dt=datetime.strptime(dt_str,"%Y-%m-%d %H:%M")
            
            x_timestamp = Timestamp()
            x_timestamp.FromDatetime(dt)
            micro_x =x_timestamp.ToMicroseconds()
            
            return micro_x
        else:
            return np.nan
    #        
    
    for d in datetimeCols:
        #Check whick column contain null value if so, convert float64 to int 32"
        dfUpsert[d]=dfUpsert[d].apply(convert_string_to_datetime_timestamp_microseconds)
        dfUpsert[d]=dfUpsert[d].astype('Int64')
        
# #         # handle null datetime value(workaround) for 2.0 but it seem to support in 3.0
        # 1970-01-01 00:00:00 UTC
        dfUpsert[d] = dfUpsert[d].fillna(0)
        
dfUpsert.info()
dfUpsert

Convert strng to datetime and microseconds accordingly
<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   incident_id     20 non-null     int64 
 1   inventory_id    20 non-null     int64 
 2   incident_type   20 non-null     object
 3   service_type    20 non-null     object
 4   severity        20 non-null     object
 5   status          20 non-null     object
 6   open_datetime   20 non-null     Int64 
 7   close_datetime  20 non-null     Int64 
 8   update_at       20 non-null     int64 
 9   _CHANGE_TYPE    20 non-null     object
dtypes: Int64(2), int64(3), object(5)
memory usage: 1.8+ KB


,incident_id,inventory_id,incident_type,service_type,severity,status,open_datetime,close_datetime,update_at,_CHANGE_TYPE
0,4372,17269,Controller/Node Failure,Incident,Major,In Progress,1705299360000000,0,1705500330000000,UPSERT
1,4097,17974,General Incident,Incident,Minor,Closed,1701060540000000,1701843960000000,1705500330000000,UPSERT
2,4198,17974,General Incident,Incident,Minor,Closed,1702019520000000,1702579320000000,1705500330000000,UPSERT
3,4274,12302,Maintenance System,Request,Cosmetic,On Hold,1702216860000000,0,1705500330000000,UPSERT
4,4305,375,Memory Failure,Incident,Minor,Closed,1703474820000000,1703498400000000,1705500330000000,UPSERT
5,4333,18123,Hard Disk Drive Failure,Incident,Major,Closed,1704423600000000,1704866400000000,1705500330000000,UPSERT
6,4332,18123,Hard Disk Drive Failure,Incident,Major,Closed,1704423600000000,1704866400000000,1705500330000000,UPSERT
7,4374,18986,Upgrade Software,Request,Cosmetic,Closed,1705323600000000,1705352400000000,1705500330000000,UPSERT
8,4311,16441,General Incident,Request,Cosmetic,Closed,1704362700000000,1704365400000000,1705500330000000,UPSERT
9,4382,13172,Software,Request,Major,In Progress,1705376280000000,0,1705500330000000,UPSERT


# Delete dataframe tranformation

In [26]:
dfDelete=dfDelete[[view_name_id,"_CHANGE_TYPE"]]
dfDelete.info()
dfDelete

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   incident_id   0 non-null      int64 
 1   _CHANGE_TYPE  0 non-null      object
dtypes: int64(1), object(1)
memory usage: 0.0+ bytes


,incident_id,_CHANGE_TYPE


# Write Json File to ingest to BQ by Buffer Protocol

In [27]:
if  dfUpsert.empty==False:
    upsert_json_file="incident_upsert.json"
    print(upsert_json_file)
    json_file_path=os.path.join(upsert_json_file)

    json_incident_data = json.loads(dfUpsert.to_json(orient = 'records'))
    with open(upsert_json_file, "w") as outfile:
        json.dump(json_incident_data, outfile)

incident_upsert.json


In [28]:
if  dfDelete.empty==False:
    delete_json_file="incident_delete.json"
    print(delete_json_file)
    json_file_path=os.path.join(delete_json_file)
    json_incident_data = json.loads(dfDelete.to_json(orient = 'records'))
    with open(json_file_path, "w") as outfile:
        json.dump(json_incident_data, outfile)


# BufferProto to BQ 

In [29]:
def get_data_pb2(view_name):
    
    x_data_pb2=None
    if view_name == "xyz_incident": 
        x_data_pb2=pb2_incident.IncidentData()
    else:
        raise Exception("No specified view name to get data pb2")
        
    return x_data_pb2

In [30]:
listColumns= df.columns.tolist()
print(listColumns)

['incident_id', 'inventory_id', 'incident_type', 'service_type', 'severity', 'status', 'open_datetime', 'close_datetime', 'update_at', '_CHANGE_TYPE']


In [31]:
pb2_data=get_data_pb2(view_name)
print(pb2_data.DESCRIPTOR)

In [32]:
if os.path.exists(upsert_json_file):
    print("OK")
    x.write_json_to_bq(listColumns=listColumns,
                              x_data_pb2=pb2_data,
                              json_data_file=upsert_json_file,
                              project_name=projectId,
                              dataset_name=main_dataset_id,
                              table_name=table_name)
                              



OK
Rows to table: 'projects/pongthorn/datasets/SMartDataAnalytics/tables/incident' have been written.
